build an LSTM model to learn sequences of words from email data. We will use this model to predict the next word.

# Predict the next word based on the sequence of words or sentences

In [8]:
import pandas as pd

In [9]:
file_content = pd.read_csv(r'C:\Users\baban\Python Code\Python\MINI PROJECT\NLP\spam.csv', encoding = "ISO-8859-1")

In [10]:
# Just selecting emails and connverting it into list
Email_Data = file_content[[ 'v2']]

In [11]:
list_data = Email_Data.values.tolist()

In [12]:
import numpy as np
import random
import pandas as pd
import sys
import os
import time
import codecs
import collections
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()

In [13]:
#Converting list to string
from collections import Iterable
def flatten(items):
    """Yield items from any nested iterable"""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x,(str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
                yield x

In [14]:
TextData=list(flatten(list_data))
TextData =''.join(TextData)

In [15]:
# Remove unwanted lines and converting into lower case
TextData = TextData.replace('\n','')
TextData = TextData.lower()

pattern = r'[^a-zA-z0-9\s]'
TextData = re.sub(pattern, '', ''.join(TextData))

In [16]:
# Tokenizing
tokens = tokenizer.tokenize(TextData)
tokens = [token.strip() for token in tokens]

In [17]:
# get the distinct words and sort it
word_counts = collections.Counter(tokens)
word_c = len(word_counts)

print(word_c)
distinct_words = [x[0] for x in word_counts.most_common()]
distinct_words_sorted = list(sorted(distinct_words))

13345


In [18]:
# Generate indexing for all words
word_index = {x: i for i, x in enumerate(distinct_words_sorted)}
# decide on sentence length
sentence_length = 25

In [19]:
# prepare the dataset of input to output pairs encoded as integers
# Generate the data for the model


# input = the input sentence to the model with index
# output = output of the model with index

InputData = []
OutputData = []
for i in range(0, word_c - sentence_length, 1):
    X = tokens[i:i + sentence_length]
    Y = tokens[i + sentence_length]
    InputData.append([word_index[char] for char in X])
    OutputData.append(word_index[Y])


In [20]:
print (InputData[:1])
print ("\n")
print(OutputData[:1])

[[5086, 12190, 6352, 9096, 3352, 1920, 8507, 5937, 2535, 7886, 5214, 12910, 6541, 4104, 2531, 2997, 11473, 5170, 1595, 12552, 6590, 6316, 12758, 12087, 8496]]


[4292]


In [21]:
# Generate X
X = numpy.reshape(InputData, (len(InputData), sentence_length, 1))
# One hot encode the output variable
Y = np_utils.to_categorical(OutputData)

In [22]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [23]:
#define the checkpoint
file_name_path="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_name_path, monitor='loss',verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [22]:
#fit the model
model.fit(X, Y, epochs=15, batch_size=128, callbacks=callbacks) 


Epoch 1/15
13320/13320 [==============================] - 39s 3ms/step - loss: 7.8639

Epoch 00001: loss improved from inf to 7.86393, saving model to weights-improvement-01-7.8639.hdf5
Epoch 2/15
13320/13320 [==============================] - 31s 2ms/step - loss: 7.1548

Epoch 00002: loss improved from 7.86393 to 7.15483, saving model to weights-improvement-02-7.1548.hdf5
Epoch 3/15
13320/13320 [==============================] - 34s 3ms/step - loss: 7.0207

Epoch 00003: loss improved from 7.15483 to 7.02071, saving model to weights-improvement-03-7.0207.hdf5
Epoch 4/15
13320/13320 [==============================] - 33s 2ms/step - loss: 6.8878

Epoch 00004: loss improved from 7.02071 to 6.88779, saving model to weights-improvement-04-6.8878.hdf5
Epoch 5/15
13320/13320 [==============================] - 31s 2ms/step - loss: 6.7928

Epoch 00005: loss improved from 6.88779 to 6.79284, saving model to weights-improvement-05-6.7928.hdf5
Epoch 6/15
13320/13320 [=============================

In [27]:
# load the network weights
try:
    file_name = "weights-improvement-15-6.5199.hdf5"
    model.load_weights(file_name)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
except OSError:
    print(" You forgot to get the updated file. Look for .hdf5  extension file in your working directory")
 

In [28]:
# Generating random sequence
start = numpy.random.randint(0, len(InputData))
input_sent = InputData[start]
# Generate index of the next word of the email
X = numpy.reshape(input_sent, (1, len(input_sent), 1))

In [29]:
predict_word = model.predict(X, verbose=0)
index = numpy.argmax(predict_word)
print(input_sent)
print ("\n")
print(index)


[667, 8859, 9106, 12187, 184, 7781, 3604, 5403, 1221, 8018, 3581, 6105, 5696, 13207, 4267, 13273, 4953, 13171, 10692, 6213, 8153, 2132, 5937, 11911, 2571]


5849


In [30]:
# Convert these indexes back to words
word_index_rev = dict((i, c) for i, c in enumerate(tokens))
result = word_index_rev[index]
sent_in = [word_index_rev[value] for value in input_sent]

In [35]:
print(' '.join(sent_in),"\n------ Next word is ------\n{}".format(result))

let okits or me colour afternoon same be me that msg busetopmessagesome to are thats verify with have from minutes sunscreen the would credits new 
------ Next word is ------
shut


'c:\\Users\\baban\\Personal-Projects'